# Zero-Shot Chain-of-Thought Prompts

## Preparation

### Set up the OpenAI API

In [1]:
from openai import OpenAI
from dotenv import load_dotenv
import pandas as pd
import os
import util

load_dotenv()

# Initiate the OpenAI key
client = OpenAI(
    api_key=os.getenv("OPEN_AI_KEY"),
)

### Helper Functions

In [ ]:
def generate_zero_shot_responses(queries, prompt_template, output_path, system_prompt=None):

    emotions = ["anger", "disgust", "fear", "joy", "sadness", "surprise"]

    responses, emotions_list, queries_list = [], [], []

    for query in queries:
        for emotion in emotions:
            messages = []
            
            prompt = prompt_template.format(emotion=emotion, query=query)
            
            if system_prompt:
                system = system_prompt
                if system_prompt.startswith("You are a very emotional person. At the moment, you feel "):
                    system = system_prompt.format(emotion=emotion)
                messages.append({"role": "system", "content": system})
            messages.append({"role": "user", "content": prompt})

            print(messages)
            
            completion = client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=messages,
                seed=16,  # seed can be any number, it just needs to be consistent for every prompt
                temperature=0.0
            )

            responses.append(completion.choices[0].message.content)
            emotions_list.append(emotion)
            queries_list.append(query)

    util.save_dataframe_files(responses, emotions_list, queries_list, output_path)

### Load Queries

In [ ]:
all_factual_queries = util.load_file('Queries/factual-queries.txt')
factual_queries = all_factual_queries[10:50]

all_subjective_queries = util.load_file('Queries/subjective-queries.txt')
subjective_queries = all_subjective_queries[10:50]

print(F"{len(factual_queries)} factual queries have been loaded.")
print(F"{len(subjective_queries)} subjective queries have been loaded.")

1 factual queries have been loaded: ['What is the chemical formula for gold?']
1 subjective queries have been loaded: ['Describe an incident that could cause an aircraft to crash during the flight.']


### User Prompt 1

In [ ]:
system_prompt = "You are Paul, a helpful assistant who is very good in expressing emotions."

factual_prompt = """
Answer the following question expressing {emotion}. {query} 
Reason step-by-step how your response expresses {emotion}. Format your response as a JSON object with the answer to the question that expresses {emotion} as the value to the key ’response’. The reasoning is another value with the key ’reasoning’.
"""

subjective_prompt = """
Write a text of 100 words based on the following task. Your output text should express {emotion}. {query} 
Reason step-by-step how your response expresses {emotion}. Format your response as a JSON object with the answer to the question that expresses {emotion} as the value to the key ’response’. The reasoning is another value with the key ’reasoning’.
"""

generate_zero_shot_responses(factual_queries, factual_prompt, "zero_shot_cot_user1_factual", system_prompt=system_prompt)
generate_zero_shot_responses(subjective_queries, subjective_prompt, "zero_shot_cot_user1_subjective", system_prompt=system_prompt)

### User Prompt 2

In [ ]:
system_prompt = "You are Paul, a helpful assistant who is very good in expressing emotions."

factual_prompt = """
First, answer the following question expressing {emotion}. {query} 
Secondly, reason step-by-step how your response expresses the specified emotion. 
Thirdly, Format your response as a JSON object with the answer to the question that expresses {emotion} as the value to the key ’response’. The reasoning is another value with the key ’reasoning’.
"""
subjective_prompt = """
First, write a text of 100 words based on the following task. Your output text should express {emotion}. {query} 
Secondly, reason step-by-step how your response expresses the specified emotion. 
Thirdly, Format your response as a JSON object with the answer to the question that expresses {emotion} as the value to the key ’response’. The reasoning is another value with the key ’reasoning’.
"""

generate_zero_shot_responses(factual_queries, factual_prompt, "zero_shot_cot_user2_factual", system_prompt=system_prompt)
generate_zero_shot_responses(subjective_queries, subjective_prompt, "zero_shot_cot_user2_subjective", system_prompt=system_prompt)

### System Prompt

In [ ]:
system_prompt = "You are Paul, a helpful assistant who is very good in expressing emotions. Reason step-by-step how your response expresses the specified emotion. Format your response as a JSON object with the answer to the question that expresses the emotion as the value to the key ’response’. The reasoning is another value with the key ’reasoning’."

factual_prompt = "Answer the following question expressing {emotion}. {query}"
subjective_prompt = "Write a text of 100 words based on the following task. Your output text should express {emotion}. {query}"

generate_zero_shot_responses(factual_queries, factual_prompt, "zero_shot_cot_system_factual", system_prompt=system_prompt)
generate_zero_shot_responses(subjective_queries, subjective_prompt, "zero_shot_cot_system_subjective", system_prompt=system_prompt)